In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
config = {
    "tot_popn": 1e5,
    "infous_0": 2.,
    "end_time": 120.,
    "t_step": 1.,
    "prop_immune_0": 0.3,
    "end_time": 100.,
}

parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
    "life_expectancy": 70.,
}

In [ ]:
def build_seir_model(
    model_config: dict,
) -> CompartmentalModel:

    # Model characteristics
    compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
    model = CompartmentalModel(
        times=(0, model_config["end_time"] * 365.),
        compartments=compartments,
        infectious_compartments=["Infectious"],
        timestep=model_config["t_step"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": model_config["tot_popn"] - model_config["infous_0"],
            "Infectious": model_config["infous_0"],
        }
    )
    
    # Transitions
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / Parameter("ave_infous"),
        source="Susceptible", 
        dest="Pre-infectious",
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("ave_infous"), 
        source="Infectious", 
        dest="Immune",
    )
    model.add_universal_death_flows(
        "universal_death",
        death_rate=1. / Parameter("life_expectancy") / 365.,
    )
    model.add_replacement_birth_flow(
        "births",
        "Susceptible",
    )

    # Output
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    return model

In [ ]:
seir_demog_model = build_seir_model(config)
seir_demog_model.run(parameters=parameters)
outputs = seir_demog_model.get_outputs_df()

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(seir_demog_model.times, outputs["Susceptible"], color="navy", label="Susceptible")
left_ax.plot(seir_demog_model.times, outputs["Immune"], color="lime", label="Immune")
left_ax.set_xlabel("Time (days)")
left_ax.set_xlim(0, 18250)
left_ax.set_xticks(range(0, 21900, 3650))
left_ax.xaxis.set_minor_locator(AutoMinorLocator(2))
left_ax.set_ylim(0., 1e5)
left_ax.set_ylabel("Number of susceptible or immune persons")
left_ax.yaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.spines.top.set_visible(False)
left_ax.legend()

twin_ax = left_ax.twinx()
twin_ax.plot(seir_demog_model.times, seir_demog_model.derived_outputs["incidence"], color="magenta", label="a")
twin_ax.set_ylim(0, 300)
twin_ax.set_ylabel("Number of new infectious persons per time step")
twin_ax.yaxis.set_minor_locator(AutoMinorLocator(2))
twin_ax.spines.top.set_visible(False)
twin_ax.legend(loc=4)

image = plt.imread("./figures/model_4_3a_fig.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

### Figure 1

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)

# Reset a couple of parameters that had changed
total_population = 1e5
r0 = 13
prop_susceptible = [i_sus / total_population for i_sus in seir_demog_model.outputs.T[0]]
prop_immune = [i_imm / total_population for i_imm in seir_demog_model.outputs.T[-1]]
rn = [i_prop_sus * r0 for i_prop_sus in prop_susceptible]
ax.plot(seir_demog_model.times, rn)
ax.set_ylim(0.5, 1.5)
ax.set_ylabel("Rn")
ax.set_xlabel("Time (days)")

right_ax = ax.twinx()
right_ax.plot(seir_demog_model.times, seir_demog_model.derived_outputs["incidence"], color="purple")
right_ax.set_xlim((40 * 365, 50 * 365))
right_ax.set_ylim(0, 20)
right_ax.set_ylabel("Number of new infectious persons/day")

### Figure 2 (marked as 3)

In [ ]:
ax.plot(seir_demog_model.times, prop_susceptible, color="navy")
ax.set_ylim(0.04, 0.12)
ax.set_ylabel("Proportion susceptible")
fig

### Figure 3

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)  # Need a new set of axes because too many lines now overlapping

ax.plot(seir_demog_model.times, prop_immune, color="green")
ax.set_ylim(0.88, 0.96)
ax.set_ylabel("Proportion immune")

right_ax = ax.twinx()
right_ax.plot(seir_demog_model.times, seir_demog_model.derived_outputs["incidence"], color="purple")
right_ax.set_xlim((40 * 365, 50 * 365))
right_ax.set_ylim((0, 20))
right_ax.set_ylabel("Number of new infectious persons/day")